<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Market-Based Valuation of Equity Options

### Option Valuation Techniques Revisited &mdash; Incomplete Markets

Dr Yves J Hilpisch

The Python Quants GmbH

**CQF Lecture, 20. September 2018, London**

## Option Valuation

Different techniques are considered:

* discounting
* replication (linear algebra)
* approximation (OLS regression)
* neural nets (learning algorithm)
* martingale pricing (probabilistic approach)

## Most Simple Financial Market

In [1]:
import numpy as np

**_Two dates only (today, tomorrow) and three future states only with equal probability._**

### Traded Financial Assets

In [2]:
# risky stock -- future payoff
S = np.array((20., 10., 5.))

In [3]:
# risky stock -- current price
S0 = 10.

In [4]:
# risk-less bond -- future payoff
B = np.array((11., 11., 11.))

In [5]:
B0 = 10.

### The Market 

In [6]:
M = np.array((S, B)).T

In [7]:
M

array([[20., 11.],
       [10., 11.],
       [ 5., 11.]])

In [8]:
M0 = np.array((S0, B0))

In [9]:
M0

array([10., 10.])

### A European Call Option 

In [10]:
# contingent claim -- future payoff
K = 15.
C = np.maximum(S - K, 0)

In [11]:
C

array([5., 0., 0.])

## Returns

In [12]:
def ret(x, x0):
    return (x.mean() / x0) - 1

In [13]:
rS = ret(S, S0)
rS

0.16666666666666652

In [14]:
rB = ret(B, B0)
rB

0.10000000000000009

## Option Valuation &mdash; Wrong Way

In [15]:
C0w1 = C.mean() / (1 + rS)
C0w1

1.4285714285714288

In [16]:
C0w2 = C.mean() / (1 + rB)
C0w2

1.5151515151515151

## Option Pricing &mdash; By Replication

In [17]:
# solving system of linear equations
# phi = np.linalg.solve(M, C)

In [18]:
# phi

In [19]:
# np.dot(M, phi)

In [20]:
# C0 = np.dot(M0, phi)

In [21]:
# C0

## Volatility

In [22]:
def vol(x, x0):
    return (x / x0).std()

In [23]:
vol(S, S0)

0.6236095644623235

In [24]:
vol(B, B0)

0.0

In [25]:
# volatility of option
# vol(C, C0)

In [26]:
# expected return of option
# ret(C, C0)

In [27]:
# C.mean() / (1 + 0.375)

## Option Pricing &mdash; By Regression

In [28]:
reg = np.linalg.lstsq(M, C, rcond=None)

In [29]:
reg

(array([ 0.35714286, -0.22727273]),
 array([1.78571429]),
 2,
 array([28.93835724,  7.11136277]))

In [30]:
p = np.dot(M, reg[0])
p

array([ 4.64285714,  1.07142857, -0.71428571])

In [31]:
np.dot(M0, reg[0])

1.2987012987012996

In [32]:
((p - C) ** 2).mean()

0.5952380952380955

## Option Pricing &mdash; By Neural Network

<img src="http://hilpisch.com/images/neural_net.png">

In [33]:
# intial weights
w = np.array((0.5, -0.5))

In [34]:
# layer one (output)
l1 = np.dot(M, w)
l1

array([ 4.5, -0.5, -3. ])

In [35]:
# deltas
d = l1 - C
d

array([-0.5, -0.5, -3. ])

In [36]:
# mean squared error
(d ** 2).mean()

3.1666666666666665

In [37]:
# learning rate
alpha = 0.001

In [38]:
# update values
u = alpha * np.dot(M.T, d)

In [39]:
u

array([-0.03 , -0.044])

In [40]:
# updating weights
w -= u

In [41]:
l1 = np.dot(M, w)
d = l1 - C
(d ** 2).mean()

2.0065560000000002

In [42]:
for _ in range(15):
    l1 = np.dot(M, w)
    d = l1 - C
    print((d ** 2).mean())
    u = alpha * np.dot(M.T, d)
    w -= u
l1

2.0065560000000002
1.843459430264
1.719769643227961
1.6088904165027007
1.5089585671614156
1.418878978445564
1.3376799411670814
1.264485950762863
1.1985078255014485
1.1390341858012527
1.085423783790188
1.037098589329643
0.9935375576632994
0.9542710114854978
0.9188755768561401


array([ 5.20556213,  0.76103229, -1.46123263])

In [43]:
np.dot(M, w)

array([ 5.17710531,  0.77672949, -1.42345842])

In [44]:
np.dot(M0, w)

1.1061518837305924

## Martingale Measure

In [45]:
from scipy.optimize import minimize

In [46]:
def E(P):
    return np.dot(S, P) / (1 + rB) 

In [47]:
P = np.array((0.3, 0.5, 0.2))

In [48]:
E(P)

10.909090909090908

In [49]:
E(P) - S0

0.9090909090909083

In [50]:
cons = ({'type': 'eq', 'fun': lambda Q: Q.sum() - 1})

In [51]:
opt = minimize(lambda Q: (E(Q) - S0) ** 2, (0.1, 0.6, 0.3), constraints=cons)

In [52]:
opt

     fun: 4.2076528774172393e-17
     jac: array([5.16188157e-06, 1.34943989e-06, 3.66844701e-07])
 message: 'Optimization terminated successfully.'
    nfev: 13
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.20714286, 0.57857143, 0.21428571])

In [53]:
Q = opt['x']

In [54]:
Q

array([0.20714286, 0.57857143, 0.21428571])

In [55]:
E(Q) - S0

6.486642334380122e-09

## Martingale Pricing

In [56]:
np.dot(C, Q) / (1 + rB)

0.9415584413160528

## Market-Based Valuation

In [57]:
from scipy.optimize import minimize

In [58]:
def E(P):
    return np.dot(C, P) / (1 + rB) # replacing S by C

In [59]:
C0 = 1.0

In [60]:
P = np.array((0.3, 0.5, 0.2))

In [61]:
E(P)

1.3636363636363635

In [62]:
E(P) - C0

0.36363636363636354

In [63]:
cons = ({'type': 'eq', 'fun': lambda Q: Q.sum() - 1})

In [64]:
opt = minimize(lambda Q: (E(Q) - C0) ** 2, (0.1, 0.6, 0.3), constraints=cons)

In [65]:
opt

     fun: 9.062456391422117e-17
     jac: array([2.21332563e-07, 0.00000000e+00, 0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 12
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.22, 0.54, 0.24])

In [66]:
Q = opt['x']

In [67]:
Q

array([0.22, 0.54, 0.24])

In [68]:
E(Q) - C0

-9.519693477955116e-09

## Martingale Pricing

In [69]:
np.dot(C, Q) / (1 + rB)

0.9999999904803065

In [70]:
put = np.maximum(K - S, 0)

In [71]:
put

array([ 0.,  5., 10.])

In [72]:
np.dot(put, Q) / (1 + rB)

4.636363650643176

In [73]:
np.dot(S, Q) / (1 + rB)

9.999999976200767

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

**Training** |
<a href="http://training.tpq.io">http://training.tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>